In [1]:
import transformers
import mlflow

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mlflow.set_tracking_uri("http://mlserver:8080")
mlflow.autolog()

2023/11/21 03:17:37 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2023/11/21 03:17:40 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/11/21 03:17:40 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.


In [19]:
import pandas as pd

eval_data = pd.DataFrame(
    {
        "inputs": [
            "What is MLflow?",
            "What is Spark?",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing  end-to-end machine learning (ML) ",
            "Apache Spark is an open-source, distributed computing system designed for big data ",
        ],
    }
)

In [18]:
conversational_pipeline = transformers.pipeline(model="microsoft/DialoGPT-medium")

with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=conversational_pipeline,
        artifact_path="chatbot"
    )

/tmp/ipykernel_322/399556978.py:4: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.34.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  model_info = mlflow.transformers.log_model(
/usr/local/lib/python3.10/dist-packages/mlflow/models/model.py:619: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.34.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2023/11/21 03:33:45 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.0a0+4136153) contains a local version label (+4136153). MLflow logged a pip requirement for this package as 'torch==2.1.0a0' without the local versio

In [7]:
    
    results = mlflow.evaluate(
        model_info.model_uri,
        eval_data,
        targets="ground_truth",
        model_type="question-answering",
    )

2023/11/21 03:21:07 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.14it/s]
2023/11/21 03:21:13 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/11/21 03:21:13 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.

No chat template is defined for this tokenizer - using the default template for the GPT2TokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right

ValueError: If using all scalar values, you must pass an index

In [5]:
    print(f"See aggregated evaluation results below: \n{results.metrics}")

    # Evaluation result for each data record is available in `results.tables`.
    eval_table = results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/tmp/ipykernel_103/3308601459.py:9: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.34.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  model_info = mlflow.transformers.log_model(
/usr/local/lib/python3.10/dist-packages/mlflow/models/model.py:619: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.34.1``. 

In [9]:
# Load the conversational pipeline as an interactive chatbot

chatbot = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

first = chatbot.predict("What is the best way to get to Antarctica?")

print(f"Response: {first}")

second = chatbot.predict("What kind of boat should I use?")

print(f"Response: {second}")

2023/11/21 03:30:08 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.88it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when 

Response: I think you can get there by boat.
Response: A boat that can go to Antarctica.


In [15]:
input = "How far to the arctic?"

In [16]:
# Load the conversational pipeline as an interactive chatbot

first = chatbot.predict(input)

print(f"Response: {first}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Response: I don't know, but I think it's a boat that can go to Antarctica.
